In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:22pt;}
div.text_cell_render.rendered_html{font-size:18pt;}
div.text_cell_render ul li pre{font-size:22pt; line-height:30px;}
div.output {font-size:22pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:22pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:22pt;padding:5px;}
table.dataframe{font-size:22px;}
</style>
"""))

<b><font size="7" color="red">ch14. 웹데이터 수집 - 동적</font></b>
# 1절. Selenium을 이용한 동적 웹크롤링 문법
- Selenium docs : https://selenium-python.readthedocs.io/
`pip install selenium`
- import에서 에러날 경우
`pip install requests`나 `conda install urllib3==1.26.18`

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

In [10]:
driver = webdriver.Chrome()
driver.get("http://www.python.org")
elem = driver.find_element(By.NAME, 'q')
elem.clear()
elem.send_keys('pycon')

In [3]:
from selenium.webdriver.common.keys import Keys
elem.send_keys(Keys.RETURN) # Enter를 입력

In [4]:
btn_elem = driver.find_element(By.CSS_SELECTOR, 'button#submit')
btn_elem.click()

In [ ]:
result_list = driver.find_elements(By.CSS_SELECTOR, 'li > h3 > a')
for result in result_list:
    title = result.text
    link  = result.get_attribute('href')
    print('{} - {}'.format(title, link))

In [6]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')
result_list = soup.select('li > h3 > a')
for result in result_list:
    print("{} - {}".format(result.text, result.attrs['href']))

PSF PyCon Trademark Usage Policy - /psf/trademarks/pycon
PyCon Italia 2016 (PyCon Sette) - /events/python-events/378/
PyCon Australia 2013 - /events/python-events/57/
PyCon AU 2019 - /events/python-events/776/
PyCon NL 2025 - /events/python-events/2084/
PyCon Australia 2014 - /events/python-events/10/
PyCon Ireland 2012 - /events/python-events/76/
PyCon Ireland 2016 - /events/python-events/429/
PyCon Ireland 2022 - /events/python-events/1320/
PyCon Australia 2014 - /events/python-events/1447/
PyCon Ireland 2024 - /events/python-events/1862/
PyCon APAC 2025 - /events/python-events/1879/
PyCon AU 2018 - /events/python-events/696/
PyCon APAC 2022 - /events/python-events/1216/
PyCon PH 2024 - /events/python-events/1661/
PyCon Ireland 2023 - /events/python-events/1568/
PyCon PL 2014 - /events/python-events/191/
PyCon MY 2015 - /events/python-events/313/
PyCon Ireland 2015 - /events/python-events/333/
PyCon AU 2015 - /events/python-events/273/


In [7]:
from urllib.parse import urlparse
current_url =driver.current_url # 셀레니윰을 통해 접근한 url
print(current_url)
parse_url = urlparse(current_url)
print(parse_url)
domain = f'{parse_url.scheme}://{parse_url.netloc}'
print(domain)

https://www.python.org/search/?q=pycon&submit=
ParseResult(scheme='https', netloc='www.python.org', path='/search/', params='', query='q=pycon&submit=', fragment='')
https://www.python.org


In [8]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
result_list = soup.select('li > h3 > a')
for result in result_list:
    print("{} - {}".format(result.text, domain+result.attrs['href']))

PSF PyCon Trademark Usage Policy - https://www.python.org/psf/trademarks/pycon
PyCon Italia 2016 (PyCon Sette) - https://www.python.org/events/python-events/378/
PyCon Australia 2013 - https://www.python.org/events/python-events/57/
PyCon AU 2019 - https://www.python.org/events/python-events/776/
PyCon NL 2025 - https://www.python.org/events/python-events/2084/
PyCon Australia 2014 - https://www.python.org/events/python-events/10/
PyCon Ireland 2012 - https://www.python.org/events/python-events/76/
PyCon Ireland 2016 - https://www.python.org/events/python-events/429/
PyCon Ireland 2022 - https://www.python.org/events/python-events/1320/
PyCon Australia 2014 - https://www.python.org/events/python-events/1447/
PyCon Ireland 2024 - https://www.python.org/events/python-events/1862/
PyCon APAC 2025 - https://www.python.org/events/python-events/1879/
PyCon AU 2018 - https://www.python.org/events/python-events/696/
PyCon APAC 2022 - https://www.python.org/events/python-events/1216/
PyCon PH 2

In [11]:
driver.close() # 브라우저 종료

# 2절. 동적웹크롤링 예제
## 2.1 다음뉴스검색

In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
driver = webdriver.Chrome()
driver.get('https://www.daum.net/')
# driver.implicitly_wait(0.5) # 최대로 0.5초동안 대기
time.sleep(0.5) # 0.5초 멈춤
driver.find_element(By.CLASS_NAME, 'tf_keyword').click()
query = input('검색할 단어는?')
driver.find_element(By.CSS_SELECTOR, 'input[type="text"]').send_keys(query)
driver.find_element(By.CLASS_NAME, 'btn_ksearch').click()

검색할 단어는?비트코인


In [15]:
# 뉴스 탭 클릭
# driver.find_elements(By.CSS_SELECTOR, 'ul.list_tab > li')[1].click()
driver.find_element(By.LINK_TEXT, '뉴스').click()

In [20]:
# 뉴스 제목과 link를 list 추가
news_list = []
strongs = driver.find_elements(By.CSS_SELECTOR, 'div.item-title > strong')
# print(len(strongs))
for strong in strongs:
    a = strong.find_element(By.TAG_NAME, 'a')
    title = a.text
    link = a.get_attribute('href')
    news_list.append([title, link])    

In [24]:
# 다음페이지로
page_nav = driver.find_element(By.CSS_SELECTOR, 'div.inner_paging')
print(page_nav.text)
next_page = page_nav.find_element(By.LINK_TEXT, "2")
next_page.click()

1 2 3 4 5


In [25]:
driver.close()

## 2.2 페이징 처리
- 다음 뉴스 페이징 처리 : 원하는 keyword를 원하는 페이지만큼 추출

In [27]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
driver = webdriver.Chrome()
driver.get('https://www.daum.net/')
# driver.implicitly_wait(0.5) # 최대로 0.5초동안 대기
time.sleep(0.5) # 0.5초 멈춤
driver.find_element(By.CLASS_NAME, 'tf_keyword').click()
query = input('검색할 단어는?')
driver.find_element(By.CSS_SELECTOR, 'input[type="text"]').send_keys(query)
driver.find_element(By.CLASS_NAME, 'btn_ksearch').click()
time.sleep(2)
# 뉴스탭 이동
driver.find_element(By.LINK_TEXT, '뉴스').click()

# 뉴스 제목과 link를 list 추가
news_list = []
pages = 3 # int(input('몇 페이지 크롤링할까?'))
for page in range(1, pages+1):
    strongs = driver.find_elements(By.CSS_SELECTOR, 'div.item-title > strong')
    # print(len(strongs))
    for strong in strongs:
        a = strong.find_element(By.TAG_NAME, 'a')
        title = a.text
        link = a.get_attribute('href')
        news_list.append([title, link])   
    page_nav = driver.find_element(By.CSS_SELECTOR, 'div.inner_paging')
    next_page = page_nav.find_element(By.LINK_TEXT, str(page+1))
    next_page.click()
    time.sleep(2)
driver.close()
import pandas as pd
pd.DataFrame(news_list, columns=['title','link']).tail(3)

검색할 단어는?비트코인


,title,link
27,"비트코인 ETF 막힌 한국…서학개미, 미국 레버리지로 '우회 매수' 급증",http://v.daum.net/v/20251111140546725
28,불장에 코인 거래대금은 연초 대비 7분의 1 뚝… 거래소 ‘1억 상금’ 유치전까지,http://v.daum.net/v/20251112154826414
29,압수된 비트코인 9조 자금세탁…40대 중국인 징역형,http://v.daum.net/v/20251112072049232


## 2.3 맞춤법 검사기
- 네이버 맞춤법 검사기 이용

In [35]:
from bs4 import BeautifulSoup

In [36]:
driver = webdriver.Chrome()
driver.get('https://www.naver.com/')
time.sleep(1)
elem = driver.find_element(By.ID, 'query')
elem.send_keys('맞춤법 검사기')
elem.send_keys(Keys.RETURN) # form태그의 submit 클릭 효과

textarea = driver.find_element(By.CLASS_NAME, 'txt_gray')
textarea.clear()
textarea.send_keys('안뇽하세요. 반갑숩니다. 감샤합니다')
btn = driver.find_element(By.CLASS_NAME, 'btn_check')
btn.click()
time.sleep(2)

soup = BeautifulSoup(driver.page_source, 'html.parser')
result = soup.select_one('p._result_text.stand_txt').text
print(result)
driver.close()

안녕하세요. 반갑습니다. 감사합니다


- 맞춤법검사전.txt 읽어 300자 단위로 짤라 list에 넣어 동적웹크롤링을 한후, 결과 출력

In [44]:
with open('data/ch14_맞춤법검사전.txt', 'r', encoding='utf-8') as f:
    text = f.read()
ready_text = [] # 300자씩 나눠진 text
while(len(text) > 300):
    temp = text[:300]
    last_dot_index = temp.rfind('\n')
    ready_text.append(text[:last_dot_index])
    text = text[last_dot_index+1 : ]
ready_text.append(text)
#print(ready_text)
print([len(ready) for ready in ready_text])
for ready in ready_text:
    print(len(ready))

[150, 173, 173]
150
173
173


In [46]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
driver = webdriver.Chrome()
time.sleep(1)
driver.get('https://www.naver.com/')
time.sleep(0.5)
elem = driver.find_element(By.ID, 'query')
elem.send_keys('맞춤법 검사기')
elem.send_keys(Keys.RETURN) # form태그의 submit 클릭 효과
time.sleep(1)
textarea = driver.find_element(By.CLASS_NAME, 'txt_gray')
results = '' # 맞춤법 검사 완료된 text
for i, ready in enumerate(ready_text):
    print(f'검사중...{i+1}/{len(ready_text)}')
    #textarea.clear()
    textarea.send_keys(Keys.CONTROL, 'a') # ctrl+a
    textarea.send_keys(ready)
    time.sleep(1)
    btn = driver.find_element(By.CLASS_NAME, 'btn_check')
    btn.click()
    time.sleep(2)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    result = soup.select_one('p._result_text.stand_txt').text
    results += result + '\n'
    time.sleep(1)
driver.close()

검사중...1/3
검사중...2/3
검사중...3/3


In [48]:
# 맞춤법 검사 결과를 파일로 출력
with open('data/ch14_맞춤법검사후.txt', 'w', encoding='utf-8') as f:
    f.write(results)

In [1]:
# 필요한 라이브러리 import
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time

# ===== 1단계: 기본 동작 테스트 =====
print("=" * 50)
print("1단계: 기본 동작 테스트")
print("=" * 50)

# Chrome 웹드라이버 실행
driver = webdriver.Chrome()
time.sleep(1)

# 네이버 메인 페이지 접속
driver.get('https://www.naver.com/')
time.sleep(1)

# 검색창에 '맞춤법 검사기' 검색
elem = driver.find_element(By.ID, 'query')
elem.send_keys('맞춤법 검사기')
elem.send_keys(Keys.RETURN)  # Enter 키로 검색 실행
time.sleep(2)

# 맞춤법 검사기 입력창 찾기
textarea = driver.find_element(By.CLASS_NAME, 'txt_gray')
textarea.clear()

# 테스트 텍스트 입력
test_text = '안뇽하세요. 반갑숩니다. 감샤합니다'
textarea.send_keys(test_text)
time.sleep(1)

# 맞춤법 검사 버튼 클릭
btn = driver.find_element(By.CLASS_NAME, 'btn_check')
btn.click()
time.sleep(2)

# 결과 크롤링
soup = BeautifulSoup(driver.page_source, 'html.parser')
result = soup.select_one('p._result_text.stand_txt').text
print(f"입력 텍스트: {test_text}")
print(f"검사 결과: {result}")
print()

# 웹드라이버 종료
driver.close()
print("1단계 완료!\n")


# ===== 2단계: 텍스트 파일 전처리 =====
print("=" * 50)
print("2단계: 텍스트 파일 전처리")
print("=" * 50)

# 파일 읽기
with open('data/ch14_맞춤법검사전.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print(f"원본 텍스트 길이: {len(text)}자")

# 300자 단위로 텍스트 분할
ready_text = []  # 분할된 텍스트를 저장할 리스트

while len(text) > 300:
    # 앞에서 300자 가져오기
    temp = text[:300]
    
    # 마지막 줄바꿈 위치 찾기
    last_newline_index = temp.rfind('\n')
    
    # 줄바꿈이 없으면 그냥 300자로 자르기
    if last_newline_index == -1:
        last_newline_index = 300
    
    # 줄바꿈 위치까지 리스트에 추가
    ready_text.append(text[:last_newline_index])
    
    # 나머지 텍스트로 업데이트
    text = text[last_newline_index + 1:]

# 남은 텍스트도 리스트에 추가
ready_text.append(text)

# 분할 결과 확인
print(f"분할된 조각 수: {len(ready_text)}개")
print("각 조각의 길이:")
for i, ready in enumerate(ready_text):
    print(f"  조각 {i+1}: {len(ready)}자")
print()
print("2단계 완료!\n")


# ===== 3단계: 자동화 프로그램 실행 =====
print("=" * 50)
print("3단계: 맞춤법 검사 자동화")
print("=" * 50)

# Chrome 웹드라이버 실행
driver = webdriver.Chrome()
time.sleep(1)

# 네이버 맞춤법 검사기 접속
driver.get('https://www.naver.com/')
time.sleep(1)

elem = driver.find_element(By.ID, 'query')
elem.send_keys('맞춤법 검사기')
elem.send_keys(Keys.RETURN)
time.sleep(2)

# 입력창 찾기
textarea = driver.find_element(By.CLASS_NAME, 'txt_gray')

# 결과를 저장할 변수
results = ''

# 모든 텍스트 조각 처리
for i, ready in enumerate(ready_text):
    print(f'검사중... {i+1}/{len(ready_text)} ({len(ready)}자)')
    
    # 기존 텍스트 지우기 (Ctrl+A로 전체 선택)
    textarea.send_keys(Keys.CONTROL, 'a')
    time.sleep(0.5)
    
    # 현재 텍스트 조각 입력
    textarea.send_keys(ready)
    time.sleep(1)
    
    # 맞춤법 검사 버튼 클릭
    btn = driver.find_element(By.CLASS_NAME, 'btn_check')
    btn.click()
    time.sleep(2)
    
    # 결과 크롤링
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    result = soup.select_one('p._result_text.stand_txt').text
    
    # 전체 결과에 추가
    results += result + '\n'
    
    time.sleep(1)

# 웹드라이버 종료
driver.close()

print()
print("맞춤법 검사 완료!")
print("=" * 50)


# ===== 4단계: 결과 파일 저장 =====
print()
print("결과 파일 저장중...")

# 맞춤법 검사 결과를 파일로 저장
with open('data/ch14_맞춤법검사후.txt', 'w', encoding='utf-8') as f:
    f.write(results)

print("저장 완료: data/ch14_맞춤법검사후.txt")
print()
print("=" * 50)
print("모든 작업이 완료되었습니다!")
print("=" * 50)

1단계: 기본 동작 테스트
입력 텍스트: 안뇽하세요. 반갑숩니다. 감샤합니다
검사 결과: 안녕하세요. 반갑습니다. 감사합니다

1단계 완료!

2단계: 텍스트 파일 전처리
원본 텍스트 길이: 1560자
분할된 조각 수: 6개
각 조각의 길이:
  조각 1: 295자
  조각 2: 279자
  조각 3: 298자
  조각 4: 281자
  조각 5: 186자
  조각 6: 216자

2단계 완료!

3단계: 맞춤법 검사 자동화
검사중... 1/6 (295자)
검사중... 2/6 (279자)
검사중... 3/6 (298자)
검사중... 4/6 (281자)
검사중... 5/6 (186자)
검사중... 6/6 (216자)

맞춤법 검사 완료!

결과 파일 저장중...
저장 완료: data/ch14_맞춤법검사후.txt

모든 작업이 완료되었습니다!


In [3]:
# 필요한 라이브러리 import
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time

print("=" * 60)
print("네이버 파파고 자동 번역 프로그램")
print("=" * 60)
print()

# ===== 1단계: 입력 파일 읽기 및 전처리 =====
print("1단계: 입력 파일 읽기 및 전처리")
print("-" * 60)

# ch14_맞춤법검사후.txt 파일 읽기
try:
    with open('data/ch14_맞춤법검사후.txt', 'r', encoding='utf-8') as f:
        text = f.read()
    print(f"✓ 파일 읽기 성공")
    print(f"✓ 원본 텍스트 길이: {len(text)}자")
except FileNotFoundError:
    print("✗ 오류: data/ch14_맞춤법검사후.txt 파일을 찾을 수 없습니다.")
    exit()

# 텍스트를 적절한 크기로 분할 (파파고는 5000자까지 가능하지만 안정성을 위해 1000자 단위로 분할)
ready_text = []  # 분할된 텍스트를 저장할 리스트
max_length = 1000  # 한 번에 번역할 최대 길이

print(f"✓ 텍스트 분할 시작 (최대 {max_length}자 단위)")

while len(text) > max_length:
    # 앞에서 max_length자 가져오기
    temp = text[:max_length]
    
    # 마지막 줄바꿈 또는 마침표 위치 찾기
    last_newline = temp.rfind('\n')
    last_period = temp.rfind('. ')
    
    # 더 뒤에 있는 위치 선택 (문장이 자연스럽게 끊기도록)
    split_pos = max(last_newline, last_period)
    
    # 적절한 위치를 못 찾았으면 그냥 max_length로 자르기
    if split_pos == -1:
        split_pos = max_length
    else:
        split_pos += 1  # 줄바꿈이나 마침표 다음 위치
    
    # 분할된 텍스트 추가
    ready_text.append(text[:split_pos].strip())
    
    # 나머지 텍스트로 업데이트
    text = text[split_pos:].strip()

# 남은 텍스트도 리스트에 추가
if text:
    ready_text.append(text)

# 분할 결과 출력
print(f"✓ 분할 완료: 총 {len(ready_text)}개 조각")
for i, chunk in enumerate(ready_text):
    print(f"  - 조각 {i+1}: {len(chunk)}자")
print()


# ===== 2단계: 웹드라이버 실행 및 파파고 접속 =====
print("2단계: 파파고 웹사이트 접속")
print("-" * 60)

# Chrome 웹드라이버 실행
print("✓ Chrome 브라우저 실행중...")
driver = webdriver.Chrome()
driver.maximize_window()  # 창 최대화
time.sleep(1)

# 파파고 웹사이트 접속
print("✓ 파파고 웹사이트 접속중...")
driver.get('https://papago.naver.com/')
time.sleep(2)

print("✓ 페이지 로딩 완료")
print()


# ===== 3단계: 자동 번역 수행 =====
print("3단계: 자동 번역 수행")
print("-" * 60)

# 번역 결과를 저장할 변수
translated_results = []

# 각 텍스트 조각을 순회하며 번역
for i, chunk in enumerate(ready_text):
    print(f"\n번역중... {i+1}/{len(ready_text)} (원문 {len(chunk)}자)")
    
    try:
        # 원문 입력 영역 찾기
        # 파파고의 원문 입력창은 id="txtSource"인 textarea
        source_textarea = driver.find_element(By.ID, 'txtSource')
        
        # 기존 텍스트 지우기
        source_textarea.click()
        time.sleep(0.5)
        source_textarea.send_keys(Keys.CONTROL, 'a')
        time.sleep(0.3)
        
        # 현재 텍스트 조각 입력
        source_textarea.send_keys(chunk)
        time.sleep(1)
        
        # 번역 버튼 클릭 (자동 번역되지만 명시적으로 클릭)
        try:
            translate_btn = driver.find_element(By.ID, 'btnTranslate')
            translate_btn.click()
        except:
            pass  # 자동 번역되는 경우 버튼이 없을 수 있음
        
        # 번역 완료 대기 (텍스트 길이에 따라 대기 시간 조정)
        wait_time = min(5, max(2, len(chunk) / 200))
        print(f"  번역 대기중... ({wait_time:.1f}초)")
        time.sleep(wait_time)
        
        # 번역 결과 추출 (여러 방법 시도)
        translated_text = ""
        
        try:
            # 방법 1: id="txtTarget"인 textarea 시도
            target_textarea = driver.find_element(By.ID, 'txtTarget')
            translated_text = target_textarea.get_attribute('value')
            if translated_text:
                print(f"  [방법1 성공] textarea value 추출")
        except:
            pass
        
        if not translated_text:
            try:
                # 방법 2: id="targetEditArea"인 div 시도
                target_div = driver.find_element(By.ID, 'targetEditArea')
                translated_text = target_div.text.strip()
                if translated_text:
                    print(f"  [방법2 성공] targetEditArea 추출")
            except:
                pass
        
        if not translated_text:
            try:
                # 방법 3: class="target_txt"인 요소 시도
                target_elements = driver.find_elements(By.CLASS_NAME, 'target_txt')
                for elem in target_elements:
                    text = elem.text.strip()
                    if text:
                        translated_text = text
                        print(f"  [방법3 성공] target_txt 추출")
                        break
            except:
                pass
        
        if not translated_text:
            try:
                # 방법 4: BeautifulSoup으로 전체 페이지 파싱
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                
                # 다양한 선택자 시도
                selectors = [
                    '#txtTarget',
                    '#targetEditArea',
                    '.target_txt',
                    '[id*="target"]',
                    '[class*="target"]'
                ]
                
                for selector in selectors:
                    element = soup.select_one(selector)
                    if element:
                        text = element.get_text(strip=True)
                        if text and len(text) > 10:  # 의미 있는 텍스트인지 확인
                            translated_text = text
                            print(f"  [방법4 성공] BeautifulSoup '{selector}' 추출")
                            break
            except:
                pass
        
        # 디버깅: 페이지 소스 일부 저장 (처음 실행시)
        if not translated_text and i == 0:
            print(f"  [디버깅] 페이지 소스를 debug_papago.html로 저장합니다.")
            with open('debug_papago.html', 'w', encoding='utf-8') as debug_file:
                debug_file.write(driver.page_source)
            print(f"  [디버깅] debug_papago.html 파일을 확인하여 번역 결과 요소를 찾아주세요.")
        
        if translated_text:
            translated_results.append(translated_text)
            print(f"  ✓ 번역 완료 (번역문 {len(translated_text)}자)")
            print(f"  미리보기: {translated_text[:50]}...")
        else:
            print(f"  ✗ 번역 결과를 찾을 수 없습니다.")
            translated_results.append(f"[번역 실패: 조각 {i+1}]")
        
        # 다음 번역을 위한 대기
        time.sleep(1)
        
    except Exception as e:
        print(f"  ✗ 오류 발생: {str(e)}")
        translated_results.append(f"[번역 오류: 조각 {i+1}]")
        continue

print()
print("✓ 모든 번역 완료!")
print()


# ===== 4단계: 웹드라이버 종료 =====
print("4단계: 브라우저 종료")
print("-" * 60)
driver.quit()
print("✓ 브라우저 종료 완료")
print()


# ===== 5단계: 번역 결과 파일 저장 =====
print("5단계: 번역 결과 저장")
print("-" * 60)

# 모든 번역 결과를 하나로 합치기 (조각 사이에 줄바꿈 추가)
final_translation = '\n\n'.join(translated_results)

# 결과 파일로 저장
try:
    with open('data/ch14_자동화영어번역본.txt', 'w', encoding='utf-8') as f:
        f.write(final_translation)
    
    print("✓ 파일 저장 성공!")
    print(f"✓ 저장 위치: data/ch14_자동화영어번역본.txt")
    print(f"✓ 총 번역문 길이: {len(final_translation)}자")
    print(f"✓ 총 조각 수: {len(translated_results)}개")
except Exception as e:
    print(f"✗ 파일 저장 오류: {str(e)}")

print()
print("=" * 60)
print("프로그램 실행 완료!")
print("=" * 60)


# ===== 결과 미리보기 =====
print("\n[번역 결과 미리보기]")
print("-" * 60)
preview_length = min(500, len(final_translation))
print(final_translation[:preview_length])
if len(final_translation) > preview_length:
    print("\n... (이하 생략) ...")
print("-" * 60)

네이버 파파고 자동 번역 프로그램

1단계: 입력 파일 읽기 및 전처리
------------------------------------------------------------
✓ 파일 읽기 성공
✓ 원본 텍스트 길이: 1557자
✓ 텍스트 분할 시작 (최대 1000자 단위)
✓ 분할 완료: 총 2개 조각
  - 조각 1: 861자
  - 조각 2: 694자

2단계: 파파고 웹사이트 접속
------------------------------------------------------------
✓ Chrome 브라우저 실행중...
✓ 파파고 웹사이트 접속중...
✓ 페이지 로딩 완료

3단계: 자동 번역 수행
------------------------------------------------------------

번역중... 1/2 (원문 861자)
  번역 대기중... (4.3초)
  [방법2 성공] targetEditArea 추출
  ✓ 번역 완료 (번역문 2044자)
  미리보기: I've already finished planting rice straws for the...

번역중... 2/2 (원문 694자)
  번역 대기중... (3.5초)
  [방법2 성공] targetEditArea 추출
  ✓ 번역 완료 (번역문 1506자)
  미리보기: He survived when he ate. Fortunately, the results ...

✓ 모든 번역 완료!

4단계: 브라우저 종료
------------------------------------------------------------
✓ 브라우저 종료 완료

5단계: 번역 결과 저장
------------------------------------------------------------
✓ 파일 저장 성공!
✓ 저장 위치: data/ch14_자동화영어번역본.txt
✓ 총 번역문 길이: 3552자
✓ 총 조각 수: 2개

프로그램 실행 완료!

[번역 결과 미리보기]
---